# Quantum Process Tomography

* **Author:** Christopher J. Wood (cjwood@us.ibm.com)
* **Last Updated:** November 8, 2018 

In [1]:
# Needed for functions
import numpy as np
import time

# Import QISKit classes
import qiskit
from qiskit import QuantumRegister, QuantumCircuit, Aer
from qiskit.tools.qi.qi import state_fidelity, outer

# Tomography functions
import sys, os
sys.path.append(os.path.abspath(os.path.join('../qiskit_ignis')))
import tomography as tomo

## 1-qubit process tomography example

In [2]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_data(circ).get('unitary')
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)
qpt_counts

{(('Zp',), ('X',)): {'0': 2000},
 (('Zp',), ('Y',)): {'0': 994, '1': 1006},
 (('Zp',), ('Z',)): {'0': 1035, '1': 965},
 (('Zm',), ('X',)): {'1': 2000},
 (('Zm',), ('Y',)): {'0': 1020, '1': 980},
 (('Zm',), ('Z',)): {'0': 1003, '1': 997},
 (('Xp',), ('X',)): {'0': 1008, '1': 992},
 (('Xp',), ('Y',)): {'0': 1005, '1': 995},
 (('Xp',), ('Z',)): {'0': 2000},
 (('Yp',), ('X',)): {'0': 994, '1': 1006},
 (('Yp',), ('Y',)): {'1': 2000},
 (('Yp',), ('Z',)): {'0': 991, '1': 1009}}

In [3]:
# Extract fitter data from tomography counts
data, basis = tomo.fitter_data(qpt_counts)

# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_mle = tomo.process_mle_fit(data, basis)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_mle / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = tomo.fitters.cvx_fit(data, basis, PSD=True, trace_preserving=True, trace=2)
choi_cvx = tomo.process_cvx_fit(data, basis)
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

MLE Fitter
fit time: 0.18453288078308105
fit fidelity: 0.9927010596006991

CVXOPT Fitter
fit time: 0.19291067123413086
fit fidelity: 0.9999597125484156


## 1-qubit process tomography of two-qubit swap gate

We will prepare qubit-0 and measure qubit-1 so the reconstructed channel should be an identity

In [6]:
# Process tomography of a Hadamard gate
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.swap(q[0], q[1])

# Ideal channel is a unitary
ideal_unitary = np.eye(2)
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
# We use the optional prepared_qubits kwarg to specify that the prepared qubit was different to measured qubit
qpt_circs = tomo.process_tomography_circuits(circ, q[1], prepared_qubits=q[0])
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)
qpt_counts

{(('Zp',), ('X',)): {'0': 988, '1': 1012},
 (('Zp',), ('Y',)): {'0': 1037, '1': 963},
 (('Zp',), ('Z',)): {'0': 2000},
 (('Zm',), ('X',)): {'0': 1009, '1': 991},
 (('Zm',), ('Y',)): {'0': 1013, '1': 987},
 (('Zm',), ('Z',)): {'1': 2000},
 (('Xp',), ('X',)): {'0': 2000},
 (('Xp',), ('Y',)): {'0': 1013, '1': 987},
 (('Xp',), ('Z',)): {'0': 986, '1': 1014},
 (('Yp',), ('X',)): {'0': 1010, '1': 990},
 (('Yp',), ('Y',)): {'0': 2000},
 (('Yp',), ('Z',)): {'0': 1000, '1': 1000}}

In [7]:
# Extract fitter data from tomography counts
data, basis = tomo.fitter_data(qpt_counts)

# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_mle = tomo.process_mle_fit(data, basis)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_mle / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = tomo.fitters.cvx_fit(data, basis, PSD=True, trace_preserving=True, trace=2)
choi_cvx = tomo.process_cvx_fit(data, basis)
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

MLE Fitter
fit time: 0.0010979175567626953
fit fidelity: 0.98941644917301

CVXOPT Fitter
fit time: 0.09900617599487305
fit fidelity: 0.9999237461111217


## 2-Qubit entangling circuit

In [8]:
# Bell-state entangling circuit
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.h(q[0])
circ.cx(q[0], q[1])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_data(circ).get('unitary')
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)

# Extract fitter data from tomography counts
data, basis = tomo.fitter_data(qpt_counts)


t = time.time()
choi_mle = tomo.process_mle_fit(data, basis)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 4, choi_mle / 4))

t = time.time()
choi_cvx = tomo.process_cvx_fit(data, basis)
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 4, choi_cvx / 4))

MLE Fitter
fit time: 0.03246784210205078
fit fidelity: 0.9857566857163759

CVXOPT Fitter
fit time: 1.426496982574463
fit fidelity: 0.9999276320532486


## Using SIC-POVM preparation basis

In [9]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_data(circ).get('unitary')
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.process_tomography_circuits(circ, q, prep_labels='SIC', prep_basis='SIC')
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)
qpt_counts

{(('S0',), ('X',)): {'0': 2000},
 (('S0',), ('Y',)): {'0': 1003, '1': 997},
 (('S0',), ('Z',)): {'0': 985, '1': 1015},
 (('S1',), ('X',)): {'0': 689, '1': 1311},
 (('S1',), ('Y',)): {'0': 993, '1': 1007},
 (('S1',), ('Z',)): {'0': 1946, '1': 54},
 (('S2',), ('X',)): {'0': 662, '1': 1338},
 (('S2',), ('Y',)): {'0': 1811, '1': 189},
 (('S2',), ('Z',)): {'0': 556, '1': 1444},
 (('S3',), ('X',)): {'0': 659, '1': 1341},
 (('S3',), ('Y',)): {'0': 181, '1': 1819},
 (('S3',), ('Z',)): {'0': 555, '1': 1445}}

In [10]:
# Extract fitter data from tomography counts
data, basis = tomo.fitter_data(qpt_counts, prep_basis='SIC')

# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_mle = tomo.process_mle_fit(data, basis)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_mle / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = tomo.process_cvx_fit(data, basis)

print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

MLE Fitter
fit time: 0.0013718605041503906
fit fidelity: 0.9934784124932428

CVXOPT Fitter
fit time: 0.0680229663848877
fit fidelity: 0.9944291012982045
